# Transaction table
“It contains money transfer and also other gifting goods and service, like you booked a ticket for others, etc.”

`TransactionDT:` timedelta from a given reference datetime (not an actual timestamp)
“TransactionDT first value is 86400, which corresponds to the number of seconds in a day (60 * 60 * 24 = 86400) so I think the unit is seconds. Using this, we know the data spans 6 months, as the maximum value is 15811131, which would correspond to day 183.”

`TransactionAMT:` transaction payment amount in USD
“Some of the transaction amounts have three decimal places to the right of the decimal point. There seems to be a link to three decimal places and a blank addr1 and addr2 field. Is it possible that these are foreign transactions and that, for example, the 75.887 in row 12 is the result of multiplying a foreign currency amount by an exchange rate?”

`ProductCD:` product code, the product for each transaction
“Product isn't necessary to be a real 'product' (like one item to be added to the shopping cart). It could be any kind of service.”

`card1 - card6:` payment card information, such as card type, card category, issue bank, country, etc.

`addr:` address
“both addresses are for purchaser
addr1 as billing region
addr2 as billing country”

`dist:` distance
"distances between (not limited) billing address, mailing address, zip code, IP address, phone area, etc.”

`P_ and (R__) emaildomain:` purchaser and recipient email domain
“ certain transactions don't need recipient, so R_emaildomain is null.”

`C1-C14:` counting, such as how many addresses are found to be associated with the payment card, etc. The actual meaning is masked.

`D1-D15:` timedelta, such as days between previous transaction, etc.

`M1-M9:` match, such as names on card and address, etc.


# Identity Table 
Variables in this table are identity information – network connection information (IP, ISP, Proxy, etc) and digital signature (UA/browser/os/version, etc) associated with transactions.
They're collected by Vesta’s fraud protection system and digital security partners.
(The field names are masked and pairwise dictionary will not be provided for privacy protection and contract agreement)

DeviceInfo : https://www.kaggle.com/c/ieee-fraud-detection/discussion/101203#583227

“`id01 to id11` are numerical features for identity, which is collected by Vesta and security partners such as device` rating, ip_domain rating, proxy rating, etc. Also it recorded behavioral fingerprint like account login times/failed to login times, how long an account stayed on the page, etc. All of these are not able to elaborate due to security partner T&C. I hope you could get basic meaning of these features, and by mentioning them as numerical/categorical, you won't deal with them inappropriately.”

[Link to Kaggle Dataset IEEE-CIS Fraud Detection](https://www.kaggle.com/competitions/ieee-fraud-detection/data?select=train_identity.csv)

In [2]:
# Import Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Connect Kaggle
!mkdir ~/.kaggle
!cp /content/drive/MyDrive/Springboard/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 1 includes product ingredients
#!kaggle datasets download -d ealaxi/paysim1
!kaggle competitions download -c ieee-fraud-detection

# Unzip file
!unzip /content/ieee-fraud-detection.zip

 99% 117M/118M [00:06<00:00, 24.1MB/s]
100% 118M/118M [00:06<00:00, 19.0MB/s]
Archive:  /content/ieee-fraud-detection.zip
  inflating: sample_submission.csv   
  inflating: test_identity.csv       
  inflating: test_transaction.csv    
  inflating: train_identity.csv      
  inflating: train_transaction.csv   


In [4]:
# import necessary packages
import pandas as pd
import numpy as np

In [5]:
# paths and reading files to dataframe
train_id_path = '/content/train_identity.csv'
train_trans_path = '/content/train_transaction.csv'

id_df = pd.read_csv(train_id_path)
trans_df = pd.read_csv(train_trans_path)

In [6]:
# data type check
print(type(id_df))
print(type(trans_df))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [7]:
# modify identity dataframe to only include DeviceType and DeviceInfo
id_df = id_df[['TransactionID','DeviceType','DeviceInfo']]

In [9]:
trans_df = trans_df[['TransactionID','isFraud','TransactionAmt','ProductCD',
                     'card1','card2','card3','card4','card5','card6',
                     'addr1','addr2',
                     'P_emaildomain','R_emaildomain']]

In [10]:
# left join to ensure that no transaction information is excluded
merged_df = pd.merge(trans_df, id_df, on='TransactionID', how='left')

In [11]:
print('DataFrame Information:')
print(merged_df.info())

DataFrame Information:
<class 'pandas.core.frame.DataFrame'>
Int64Index: 590540 entries, 0 to 590539
Data columns (total 16 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   TransactionID   590540 non-null  int64  
 1   isFraud         590540 non-null  int64  
 2   TransactionAmt  590540 non-null  float64
 3   ProductCD       590540 non-null  object 
 4   card1           590540 non-null  int64  
 5   card2           581607 non-null  float64
 6   card3           588975 non-null  float64
 7   card4           588963 non-null  object 
 8   card5           586281 non-null  float64
 9   card6           588969 non-null  object 
 10  addr1           524834 non-null  float64
 11  addr2           524834 non-null  float64
 12  P_emaildomain   496084 non-null  object 
 13  R_emaildomain   137291 non-null  object 
 14  DeviceType      140810 non-null  object 
 15  DeviceInfo      118666 non-null  object 
dtypes: float64(6), int64(3), object(7

In [12]:
# show first five rows
merged_df.head()

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,R_emaildomain,DeviceType,DeviceInfo
0,2987000,0,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,NaN,NaN,NaN,NaN
1,2987001,0,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,gmail.com,NaN,NaN,NaN
2,2987002,0,59.0,W,4663,490.0,150.0,visa,166.0,debit,330.0,87.0,outlook.com,NaN,NaN,NaN
3,2987003,0,50.0,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,87.0,yahoo.com,NaN,NaN,NaN
4,2987004,0,50.0,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,87.0,gmail.com,NaN,mobile,SAMSUNG SM-G892A Build/NRD90M


In [13]:
#check for missing values
merged_df.isnull().sum()

TransactionID          0
isFraud                0
TransactionAmt         0
ProductCD              0
card1                  0
card2               8933
card3               1565
card4               1577
card5               4259
card6               1571
addr1              65706
addr2              65706
P_emaildomain      94456
R_emaildomain     453249
DeviceType        449730
DeviceInfo        471874
dtype: int64

In [14]:
# counts and percents unique values
unique_counts = merged_df.nunique()
print(f"Unique Counts:\n{unique_counts}\n")
print(f"Proportion of Unique Counts:\n{unique_counts / len(merged_df)}\n")

Unique Counts:
TransactionID     590540
isFraud                2
TransactionAmt     20902
ProductCD              5
card1              13553
card2                500
card3                114
card4                  4
card5                119
card6                  4
addr1                332
addr2                 74
P_emaildomain         59
R_emaildomain         60
DeviceType             2
DeviceInfo          1786
dtype: int64

Proportion of Unique Counts:
TransactionID     1.000000
isFraud           0.000003
TransactionAmt    0.035395
ProductCD         0.000008
card1             0.022950
card2             0.000847
card3             0.000193
card4             0.000007
card5             0.000202
card6             0.000007
addr1             0.000562
addr2             0.000125
P_emaildomain     0.000100
R_emaildomain     0.000102
DeviceType        0.000003
DeviceInfo        0.003024
dtype: float64



In [15]:
# show the unique values for card 4
card4_counts = merged_df['card4'].value_counts()
print("Count of unique values in card4:")
print(card4_counts)

Count of unique values in card4:
visa                384767
mastercard          189217
american express      8328
discover              6651
Name: card4, dtype: int64


In [16]:
# show the unique values for card 6
card6_counts = merged_df['card6'].value_counts()
print("Count of unique values in card6:")
print(card4_counts)

Count of unique values in card6:
visa                384767
mastercard          189217
american express      8328
discover              6651
Name: card4, dtype: int64


In [17]:
# show the uniqe values for isFlaggedFraud
devicetype_counts = merged_df['DeviceType'].value_counts()
print("Count of unique values in DeviceType:")
print(devicetype_counts)

Count of unique values in DeviceType:
desktop    85165
mobile     55645
Name: DeviceType, dtype: int64


In [18]:
# descriptive statistics for the numerical data
merged_df.describe()

,TransactionID,isFraud,TransactionAmt,card1,card2,card3,card5,addr1,addr2
count,5.905400e+05,590540.000000,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000
mean,3.282270e+06,0.034990,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630
std,1.704744e+05,0.183755,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623
min,2.987000e+06,0.000000,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000
25%,3.134635e+06,0.000000,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000
50%,3.282270e+06,0.000000,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000
75%,3.429904e+06,0.000000,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000
max,3.577539e+06,1.000000,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000


In [20]:
duplicated_rows = merged_df[merged_df.duplicated()]
if duplicated_rows.empty:
    print("There are no duplicates.")
else:
    print("Duplicate Rows:")
    print(duplicated_rows)

There are no duplicates.
